# Simulation of RCT EMS v2

Want to graph:
 - Travel Times
 - Coverages

In [1]:
%matplotlib notebook

import pandas
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from sys import argv
import scipy.stats as stats
from scipy.interpolate import make_interp_spline, BSpline
import numpy as np
from numpy import median, mean
from functools import reduce
from collections import Counter
from os import listdir

import subprocess


# Execute the Simulator
## Print the results

In [2]:
commands = [
    'python3 run.py configurations/exp-scenarios/tj.yaml'
]

for c in commands:
    print(c)
    p = subprocess.Popen(c.split(' '), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    p.communicate()
    print("Done.")
    

python3 run.py configurations/exp-scenarios/tj.yaml
Done.


In [3]:
results = 'results/tijuana-multi-scenario/'
csvs = []
for f in listdir(results):
    
    path = results + f
    print(path)
    p = pandas.read_csv(path)
    csvs.append(p)


results/tijuana-multi-scenario/chosen_hospitals.csv
results/tijuana-multi-scenario/processed_cases.csv
results/tijuana-multi-scenario/metrics.csv
results/tijuana-multi-scenario/chosen_ambulances.csv
results/tijuana-multi-scenario/chosen_bases.csv


In [4]:
csvs[0]

,latitude,longitude
0,32.525069,-117.009759
1,32.502700,-117.003710
2,32.518000,-117.007800


In [5]:
csvs[1]

,id,date,latitude,longitude,priority,ambulance,start_time,TO_INCIDENT_duration,AT_INCIDENT_duration,TO_HOSPITAL_duration,AT_HOSPITAL_duration,TO_BASE_duration,OTHER_duration,hospital_latitude,hospital_longitude
0,1,2019-03-22 00:38:08.415197,32.522069,-116.966513,4,6,2019-03-22 00:38:08.415197,0 days 00:03:40.000000000,0 days 00:09:06.000000000,0 days 00:14:02.000000000,0 days 00:09:24.000000000,0 days 00:08:01.000000000,0 days 00:00:00.000000000,32.525069,-117.009759
1,2,2019-03-22 02:27:06.731925,32.520818,-117.061246,3,1,2019-03-22 02:27:06.731925,0 days 00:08:13.000000000,0 days 00:06:31.000000000,0 days 00:13:38.000000000,0 days 00:06:52.000000000,0 days 00:06:54.000000000,0 days 00:00:00.000000000,32.525069,-117.009759
2,3,2019-03-22 03:51:44.145973,32.475803,-116.909168,4,7,2019-03-22 03:51:44.145973,0 days 00:07:03.000000000,0 days 00:05:00.000000000,0 days 00:16:09.000000000,0 days 00:07:10.000000000,0 days 00:13:12.000000000,0 days 00:00:00.000000000,32.525069,-117.009759
3,4,2019-03-22 03:57:25.661029,32.507847,-116.894599,4,4,2019-03-22 03:57:25.661029,0 days 00:11:01.000000000,0 days 00:09:17.000000000,0 days 00:18:21.000000000,0 days 00:08:21.000000000,0 days 00:19:49.000000000,0 days 00:00:00.000000000,32.525069,-117.009759
4,5,2019-03-22 06:07:48.941092,32.495285,-117.019509,4,1,2019-03-22 06:07:48.941092,0 days 00:06:55.000000000,0 days 00:08:12.000000000,0 days 00:06:14.000000000,0 days 00:05:54.000000000,0 days 00:06:54.000000000,0 days 00:00:00.000000000,32.525069,-117.009759
5,6,2019-03-22 07:03:47.469302,32.499983,-116.852513,2,4,2019-03-22 07:03:47.469302,0 days 00:08:14.000000000,0 days 00:06:27.000000000,0 days 00:24:04.000000000,0 days 00:05:53.000000000,0 days 00:19:49.000000000,0 days 00:00:00.000000000,32.525069,-117.009759
6,7,2019-03-22 08:04:51.336994,32.521363,-116.943712,4,6,2019-03-22 08:04:51.336994,0 days 00:04:45.000000000,0 days 00:07:48.000000000,0 days 00:15:55.000000000,0 days 00:07:29.000000000,0 days 00:09:50.000000000,0 days 00:00:00.000000000,32.518000,-117.007800
7,8,2019-03-22 08:44:51.291772,32.468771,-116.889247,4,7,2019-03-22 08:44:51.291772,0 days 00:05:35.000000000,0 days 00:08:54.000000000,0 days 00:18:40.000000000,0 days 00:08:26.000000000,0 days 00:13:12.000000000,0 days 00:00:00.000000000,32.525069,-117.009759
8,9,2019-03-22 10:36:01.206385,32.473124,-116.980952,3,3,2019-03-22 10:36:01.206385,0 days 00:10:58.000000000,0 days 00:09:18.000000000,0 days 00:17:41.000000000,0 days 00:09:40.000000000,0 days 00:07:37.000000000,0 days 00:00:00.000000000,32.518000,-117.007800
9,10,2019-03-22 11:02:47.538965,32.471279,-116.990725,4,11,2019-03-22 11:02:47.538965,0 days 00:10:58.000000000,0 days 00:06:59.000000000,0 days 00:16:56.000000000,0 days 00:06:50.000000000,0 days 00:07:37.000000000,0 days 00:00:00.000000000,32.518000,-117.007800


In [6]:
csvs[2]

,timestamp,percent_coverage,total_delay,count_pending
0,2019-03-22 00:38:08.415197,0.63,0 days 00:00:00.000000000,0
1,2019-03-22 00:41:48.415197,0.63,0 days 00:00:00.000000000,0
2,2019-03-22 00:50:54.415197,0.63,0 days 00:00:00.000000000,0
3,2019-03-22 01:04:56.415197,0.63,0 days 00:00:00.000000000,0
4,2019-03-22 01:14:20.415197,0.63,0 days 00:00:00.000000000,0
5,2019-03-22 01:22:21.415197,0.64,0 days 00:00:00.000000000,0
6,2019-03-22 02:27:06.731925,0.64,0 days 00:00:00.000000000,0
7,2019-03-22 02:35:19.731925,0.64,0 days 00:00:00.000000000,0
8,2019-03-22 02:41:50.731925,0.64,0 days 00:00:00.000000000,0
9,2019-03-22 02:55:28.731925,0.64,0 days 00:00:00.000000000,0


In [7]:
csvs[3]

,id,base_latitude,base_longitude,capability
0,0,32.53378,-117.112,BASIC
1,1,32.52497,-117.028,BASIC
2,2,32.43844,-117.043,BASIC
3,3,32.50691,-116.964,BASIC
4,4,32.49432,-116.869,BASIC
5,5,32.53975,-116.939,BASIC
6,6,32.50802,-116.957,BASIC
7,7,32.44827,-116.885,BASIC
8,8,32.53378,-117.112,BASIC
9,9,32.52497,-117.028,BASIC


In [8]:
csvs[4]

,latitude,longitude
0,32.53378,-117.112
1,32.52497,-117.028
2,32.43844,-117.043
3,32.50691,-116.964
4,32.49432,-116.869
5,32.53975,-116.939
6,32.50802,-116.957
7,32.44827,-116.885


# Want to see number of events per hour.

In [9]:
cases = csvs[1]
dates = [datetime.strptime(arr[1], '%Y-%m-%d %H:%M:%S.%f').replace(
minute=0, second=0, microsecond=0) for arr in cases.values]


In [10]:
# matplotlib does not play well with datetimes

first = dates[0]
deltas = [(date-first).total_seconds()/3600 for date in dates]


In [11]:
c = Counter(deltas)
sum(c.values()), c 

(120,
 Counter({0.0: 1,
          2.0: 1,
          3.0: 2,
          6.0: 1,
          7.0: 1,
          8.0: 2,
          10.0: 1,
          11.0: 1,
          12.0: 3,
          13.0: 1,
          14.0: 3,
          15.0: 3,
          16.0: 1,
          17.0: 2,
          19.0: 2,
          20.0: 1,
          21.0: 1,
          22.0: 1,
          23.0: 3,
          25.0: 1,
          27.0: 1,
          35.0: 2,
          37.0: 3,
          38.0: 2,
          39.0: 3,
          41.0: 2,
          42.0: 3,
          43.0: 2,
          44.0: 1,
          45.0: 2,
          46.0: 1,
          47.0: 3,
          48.0: 1,
          51.0: 1,
          52.0: 2,
          54.0: 2,
          55.0: 2,
          56.0: 1,
          57.0: 1,
          59.0: 2,
          62.0: 2,
          64.0: 2,
          65.0: 1,
          66.0: 1,
          68.0: 2,
          69.0: 1,
          70.0: 1,
          71.0: 1,
          72.0: 2,
          76.0: 2,
          78.0: 1,
          79.0: 1,
          82

In [12]:
l = list(deltas)

plt.figure("Histogram: The number of cases per hour")
ys, xs, patches = plt.hist(deltas, bins=int((max(deltas)+1)/1), histtype='step')
plt.show()
ys = np.append(ys, [0])

plt.figure("Bar graph: The number of cases per hour")
plt.bar(c.keys(), c.values())

plt.show()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
# Smoothed the original
smooth_xs = np.linspace(int(min(c.keys())), int(max(c.keys())), 300)

spl = make_interp_spline(list(c.keys()), list(c.values()), k=3)
smooth_ys = spl(smooth_xs)


plt.figure("This is the curve you were looking for.")
plt.plot(smooth_xs, smooth_ys,)
plt.show()

plt.figure("overlay")
plt.hist(deltas, bins=int((max(deltas)+1)), histtype='step', color='black')
plt.plot(smooth_xs, smooth_ys, '-', color='red')

plt.show()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Want to see the number of cases that have been reached within r1 and r2

In [29]:
# to incident durations
durations = [pandas.to_timedelta(array[7]) for array in  cases.values]
r1 = timedelta(minutes=10)
r2 = timedelta(minutes=14)

In [34]:
times = {
    'lt_r1' : list(filter(lambda t: t <  r1, durations)),
    'lt_r2' : list(filter(lambda t: t <  r2, durations)),
    'beyond': list(filter(lambda t: t >= r2, durations)),
}

counts = {k:len(v) for k,v in times.items()}
counts

{'lt_r1': 82, 'lt_r2': 119, 'beyond': 1}

In [36]:
plt.figure("Travel times for the TO_INCIDENT events ")

plt.bar(counts.keys(), counts.values())

plt.show()

<IPython.core.display.Javascript object>

# Want to plot the histogram for the coverage

In [65]:
metrics = csvs[2]
coverages = [(pandas.to_datetime(arr[0]), float(arr[1])) for arr in metrics.values]

xs = [cov[0] for cov in coverages]
ys = [cov[1] for cov in coverages]

first = xs[0]
xs = [(date-first).total_seconds()/3600 for date in xs]

# Smoothed the original
smooth_xs = np.linspace(int(min(xs)), int(max(xs)), 300)

spl = make_interp_spline(xs, ys, k=3)
smooth_ys = spl(smooth_xs)


plt.figure("Coverages over time ")
plt.bar(xs, ys, width=0.01)
plt.plot(smooth_xs, smooth_ys, 'r--')
plt.show()

<IPython.core.display.Javascript object>

## Read data and organize

Read as a list of dicts so that the same index/object is used for each row.

In [14]:
# Read pandas CSV file 
def read_csv_as_list_of_dicts(filename):
    return 

In [15]:
# Read the specific files
filenames = [
    ""
]

## Restructure the data that is easy for the matplotlib
Want lists of xs and ys.

In [16]:
def restructure_times():
    return

In [17]:
def restructure_coverage():
    return

## Plot them. 
Want multiple plots per graph. Request asked for 1 graph, but still seems to make sense to have 4 graphs.

In [18]:
def plot_times():
    return

In [19]:
def plot_coverage():
    return